In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import os

In [2]:
path_of_saved_model = Path("./trained_model.h5")
model = tf.keras.models.load_model(path_of_saved_model)

In [3]:
path_of_saved_model

WindowsPath('trained_model.h5')

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [5]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\sunny\AppData\Local\Temp\tmpni_sag0n\assets


In [8]:
tflite_model_dir = Path("./tflite_model_dir")
tflite_model_dir.mkdir(exist_ok=True, parents=True)

In [11]:
tflite_model_file = tflite_model_dir/"vgg16_model.tflite"
tflite_model_file.write_bytes(tflite_model)

59067384

In [13]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

tflite_model_quant_file = tflite_model_dir/"vgg16_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)


INFO:tensorflow:Assets written to: C:\Users\sunny\AppData\Local\Temp\tmp9yddqq8w\assets


INFO:tensorflow:Assets written to: C:\Users\sunny\AppData\Local\Temp\tmp9yddqq8w\assets


14870960

In [14]:
14870960/59067384

0.25176263096398516

In [16]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [17]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [18]:
import matplotlib.pyplot as plt
test_img = plt.imread("./data/unzip_data_dir/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")


In [19]:
test_img.shape

(534, 800, 3)

In [22]:
interpreter.get_input_details()[0]["shape"]

array([  1, 224, 224,   3])

In [24]:
resized_img = tf.image.resize(test_img, (224, 224))



In [25]:
input_data = tf.expand_dims(resized_img, axis=0)


In [27]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [28]:
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)


In [29]:
pred

array([[1., 0.]], dtype=float32)

In [30]:
label_map = {0: 'ants', 1: 'bees'}

In [33]:
argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]

'ants'

In [35]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

interpreter_quant.set_tensor(input_index, input_data)
interpreter_quant.invoke()
pred = interpreter_quant.get_tensor(output_index)

argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]


'ants'